In [1]:
from requests import get
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
from time import time
from IPython.core.display import clear_output
from warnings import warn
import math
import string
import time
from functools import wraps
from colorama import Fore
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import selenium.common.exceptions
import datetime
import dateutil
import numpy as np
from datetime import timedelta
import sys
import dateutil.parser as parser
import os
import ast

In [2]:
def grabInflationRate(month, year):
  options = webdriver.ChromeOptions()
  options.add_argument("no-sandbox")
  options.add_argument("disable-dev-shm-usage")
  #   GIVE ADDITIONAL SPACE FOR DISK CACHE
  options.add_argument("disk-cache-size=53687091200")
  #options.add_argument("headless")
  options.add_argument("user-data-dir=C:/Users/Brian/AppData/Local/Google/Chrome/User Data/Thread_One")
  driver = webdriver.Chrome("C:\\Users\Brian\Desktop\chromedriver.exe" , chrome_options=options)
  driver.get('https://data.bls.gov/cgi-bin/cpicalc.pl')
  
  time.sleep(1)
  cost = driver.execute_script("document.getElementById('cost1').value = '1.00'")
  
  month_one = "document.getElementById('year1-month').value = '{}';".format(month)
  driver.execute_script(month_one)
  year_one = "document.getElementById('year1-year').value = '{}';".format(year)
  driver.execute_script(year_one)
  calculate_btn = "document.getElementById('submit').click();"
  driver.execute_script(calculate_btn)

  
  time.sleep(3)
  answer = driver.execute_script("return document.getElementById('answer').innerHTML")
  answer = answer[1:]
  
  temp = pd.DataFrame({
    'year': [year],
    'month': [month],
    'value': [answer]
  })
  driver.quit()
  
  print("Year: {}".format(year))
  print("Month: {}".format(month))
  print("Value: {}".format(answer))
  clear_output(wait=True)
  return temp


In [3]:
inflation_rate = pd.DataFrame({
  'year': [],
  'month': [],
  'value': []
})

In [4]:
months = ['01','02','03','04','05','06','07','08','09','10','11','12']
for year in range(1900, 2020, 1):
  year = str(year)
#   if year == '2019':
#     for month in months[:8]:
#       value = grabInflationRate(month, year)
#       inflation_rate = inflation_rate.append(value, ignore_index=True)
#   else:
  for month in months:
    value = grabInflationRate(month, year)
    inflation_rate = inflation_rate.append(value, ignore_index=True)

Year: 2019
Month: 12
Value: 1.01


In [5]:
inflation_rate

,year,month,value
0,1980,01,3.32
1,1980,02,3.28
2,1980,03,3.23
3,1980,04,3.19
4,1980,05,3.16
...,...,...,...
475,2019,08,1.01
476,2019,09,1.01
477,2019,10,1.01
478,2019,11,1.01


In [6]:
inflation_rate.to_csv('../../data/1900-2019/inflation_rate.csv')

In [ ]:
'''
  CREATE DATAFRAME
'''

def createDataFrame():
  df = pd.read_csv("../data/complete_dataset.csv")

  '''
    REMOVE MOVIES IN DATASET BEFORE 1980
  '''
  # DROP NA COLUMNS
  for idx, row in enumerate(df.itertuples()):
    if type(row.release)!=str or type(row.directors)!=str or type(row.cast)!=str or type(row.mpaa)!=str or type(row.genre)!=str:
      df = df.drop(idx)

  new_release= []
  for date in df['release']:
    if type(date)!=str:
      new_release.append("null")
    elif int(date[-4:]) < 1980:
      new_release.append("null")
    else:
      new_release.append(date)
  df['release'] = new_release

  # CONVERT DATE TO ISO FORMAT
  new_dates = []
  for date in df['release']:
    if date in 'null':
      new_dates.append("null")
    else:
      new_dates.append(dateutil.parser.parse(date).strftime('%Y-%m-%d'))

  df['date'] = new_dates

  # CLEAN DATES TO HAVE YYYY/MM/DD
  new_dates = []
  for date in df['date']:
    if len(date) < 10:
      new_dates.append("null")
    else:
      new_dates.append(date)
  df['date'] = new_dates
  df = df.replace("null", np.nan)
  df = df.dropna(subset=['dom_box_office', 'date', 'cast', 'cinematographer'])
  
  
  
  # ADD FIRST_BILLED COLUMN
  first_billed = []
  for row in df.itertuples():
    cast = ast.literal_eval(row.cast)
    cast = cast[:15]
    first_billed.append(cast)
  df['first_billed'] = first_billed
  
  #ADD MAIN STUDIO COLUMN
  main_prods = []
  for comp in df['studio']:
    if type(comp)!=str:
      main_prods.append(np.nan)
    else:
      studio = ast.literal_eval(comp)
      studio = studio[0]
      main_prods.append(studio)
  df['main_studio'] = main_prods
  #ADD MAIN SPECIAL EFFECTS COLUMN
  main_specs = []
  for val in df['special_effects']:
    if type(val)!=str:
      main_specs.append(np.nan)
    else:
      spec = ast.literal_eval(val)
      spec = spec[0]
      main_specs.append(spec)
  df['main_effects'] = main_specs
#   CREATE MAIN_GENRE CATEGORY
  main_genres = []
  for row in df.itertuples():
    genres = ast.literal_eval(row.genre)
    main_genre = genres[0]
    main_genres.append(main_genre)
  df['main_genre'] = main_genres
  # CREATE YEAR COLUMN
  years = []
  for date in df['release']:
      years.append(date[-4:])
  df['year'] = years
  df.drop(df[df['year'] == 'null'].index)
  datetimes = []
  for date in df['date']:
    date = date.split("-")
    date = datetime.datetime(int(date[0]), int(date[1]), int(date[2]))
    datetimes.append(date)
  df['datetime'] = datetimes
  return df

df = createDataFrame()
pd.set_option('display.max_columns', 100)
df.head(5)